<a href="https://colab.research.google.com/github/janimiyarj/Bert-Model/blob/main/Bert_Practise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # TensorFlow Text for preprocessing
import numpy as np


In [5]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")


In [10]:
X_train = np.array(["I love this movie", "I hate this movie", "This was an awesome experience",
                    "It was a terrible day", "I enjoyed the film", "This is a bad product"])
y_train = np.array([1, 0, 1, 0, 1, 0])

X_test = np.array(["I like this", "I dislike this", "Great work!", "Bad experience"])
y_test = np.array([1, 0, 1, 0])


In [2]:
pip install tensorflow

In [3]:
pip install tensorflow_text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 54.5 MB/s eta 0:00:00


In [7]:
X_train = ["I love this movie", "I hate this movie", "This was an awesome experience",
           "It was a terrible day", "I enjoyed the film", "This is a bad product"]

y_train = [1, 0, 1, 0, 1, 0]

X_test = ["I like this", "I dislike this", "Great work!", "Bad experience"]

y_test = [1, 0, 1, 0]


In [17]:
pip install transformers

In [19]:
pip install datasets


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00


In [27]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset

# Example dataset for binary classification
X_train = ["I love this movie", "I hate this movie", "This was an awesome experience",
           "It was a terrible day", "I enjoyed the film", "This is a bad product"]
y_train = [1, 0, 1, 0, 1, 0]

X_test = ["I like this", "I dislike this", "Great work!", "Bad experience"]
y_test = [1, 0, 1, 0]

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode the training data
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128)

# Create datasets using the Hugging Face `datasets` library
train_dataset = Dataset.from_dict({'input_ids': train_encodings['input_ids'],
                                    'attention_mask': train_encodings['attention_mask'],
                                    'labels': y_train})

test_dataset = Dataset.from_dict({'input_ids': test_encodings['input_ids'],
                                   'attention_mask': test_encodings['attention_mask'],
                                   'labels': y_test})

# Load BERT for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

# Make predictions
predictions = trainer.predict(test_dataset)
y_predicted = np.argmax(predictions.predictions, axis=1)

print("Predicted classes:", y_predicted)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.789772
2,No log,0.788898
3,No log,0.787604
4,0.694800,0.786988
5,0.694800,0.787202
6,0.694800,0.788040
7,0.628600,0.787769
8,0.628600,0.788150
9,0.628600,0.788139
10,0.573500,0.787327


Predicted classes: [0 1 0 0]


In [24]:
print(test_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 4
})


In [25]:
import pandas as pd

df = test_dataset.to_pandas()  # Convert to DataFrame
print(df)  # Print the DataFrame


                       input_ids   attention_mask  labels
0   [101, 1045, 2066, 2023, 102]  [1, 1, 1, 1, 1]       1
1  [101, 1045, 18959, 2023, 102]  [1, 1, 1, 1, 1]       0
2    [101, 2307, 2147, 999, 102]  [1, 1, 1, 1, 1]       1
3      [101, 2919, 3325, 102, 0]  [1, 1, 1, 1, 0]       0


In [26]:
# Print the first example in the dataset
print("First example:")
print(test_dataset[0])

# Print the first two examples
print("\nFirst two examples:")
print(test_dataset[:2])

# Convert to a Pandas DataFrame and print it
import pandas as pd

df = test_dataset.to_pandas()
print("\nDataset as a DataFrame:")
print(df)

# Select and print specific rows
selected_rows = test_dataset.select([0, 1, 2])
print("\nSelected rows:")
print(selected_rows)


First example:
{'input_ids': [101, 1045, 2066, 2023, 102], 'attention_mask': [1, 1, 1, 1, 1], 'labels': 1}

First two examples:
{'input_ids': [[101, 1045, 2066, 2023, 102], [101, 1045, 18959, 2023, 102]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]], 'labels': [1, 0]}

Dataset as a DataFrame:
                       input_ids   attention_mask  labels
0   [101, 1045, 2066, 2023, 102]  [1, 1, 1, 1, 1]       1
1  [101, 1045, 18959, 2023, 102]  [1, 1, 1, 1, 1]       0
2    [101, 2307, 2147, 999, 102]  [1, 1, 1, 1, 1]       1
3      [101, 2919, 3325, 102, 0]  [1, 1, 1, 1, 0]       0

Selected rows:
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3
})
